# Generating Music

In [ ]:
""" This module generates notes for a midi file using the
    trained neural network """
import pickle
import numpy
from music21 import instrument, note, stream, chord
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import BatchNormalization as BatchNorm
from keras.layers import Activation

In [ ]:

def generate():
    """ Generate a piano midi file """
    #load the notes used to train the model
    with open('data/notes', 'rb') as filepath:
        notes = pickle.load(filepath)

    # Get all pitch names
    pitchnames = sorted(set(item for item in notes))
    # Get all pitch names
    n_vocab = len(set(notes))

    network_input, normalized_input = prepare_sequences(notes, pitchnames, n_vocab)
    model = create_network(normalized_input, n_vocab)
    prediction_output = generate_notes(model, network_input, pitchnames, n_vocab)
    create_midi(prediction_output)


In [ ]:

def prepare_sequences(notes, pitchnames, n_vocab):
    """ Prepare the sequences used by the Neural Network """
    # map between notes and integers and back
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

    sequence_length = 100
    network_input = []
    output = []
    for i in range(0, len(notes) - sequence_length, 1):
        sequence_in = notes[i:i + sequence_length]
        sequence_out = notes[i + sequence_length]
        network_input.append([note_to_int[char] for char in sequence_in])
        output.append(note_to_int[sequence_out])

    n_patterns = len(network_input)

    # reshape the input into a format compatible with LSTM layers
    normalized_input = numpy.reshape(network_input, (n_patterns, sequence_length, 1))
    # normalize input
    normalized_input = normalized_input / float(n_vocab)

    return (network_input, normalized_input)

In [ ]:
# 학습된 신경망을 사용하여 음악을 생성하려면 이전과 같은 상태로 만들어야 합니다
# 간단하게 하기 위해 학습 섹션의 코드(chord)를 재사용하여 데이터를 준비하고 전과 동일한 방식으로 네트워크 모델을 설정합니다.


def create_network(network_input, n_vocab):
    """ create the structure of the neural network """
    model = Sequential()
    model.add(LSTM(
        512,
        input_shape=(network_input.shape[1], network_input.shape[2]),
        recurrent_dropout=0.3,
        return_sequences=True
    ))
    model.add(LSTM(512, return_sequences=True, recurrent_dropout=0.3,))
    model.add(LSTM(512))
    model.add(BatchNorm())
    model.add(Dropout(0.3))
    model.add(Dense(256))
    model.add(Activation('relu'))
    model.add(BatchNorm())
    model.add(Dropout(0.3))
    model.add(Dense(n_vocab))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

    # Load the weights to each node
    model.load_weights('weights.hdf5')
    # 각각의 뉴런(노드)의 가중치를 로드합니다.
    # 파일에 저장한 학습 결과를 가져오는 것과 같습니다!
    
    # instead of training the network we load the weights that we saved during the training section into the model.
    # 네트워크를 교육하는 대신 학습 섹션에서 저장한 가중치를 모델에 로드합니다.
    
    return model



# Generating Notes

In [ ]:
# 이제 학습된 모델을 사용하여 노트 생성을 시작할 수 있습니다.
# 우리가 매번 다른 시퀀스를 입력으로 준다면, 아무것도 하지 않고도 매번 다른 결과를 얻을 수 있습니다. 
# 매번 다른 시퀀스를 입력하기 위해서는 랜덤 함수를 이용하면 됩니다!


def generate_notes(model, network_input, pitchnames, n_vocab):
    """ Generate notes from the neural network based on a sequence of notes """
    # pick a random sequence from the input as a starting point for the prediction
    # 입력 시퀀스를 랜덤하게 주는 부분.
    start = numpy.random.randint(0, len(network_input)-1)
    
    # 네트워크 출력을 디코딩하는 매핑 기능도 생성해야 합니다.
    # 입력은 categorical 한 것을 숫자로 바꾸었지만 이번엔 반대로 숫자에서 categorical 데이터(정수에서 노트까지)로 매핑합니다.
    # 숫자를 노트로 매핑하는 사전을 생성합니다.
    int_to_note = dict((number, note) for number, note in enumerate(pitchnames))

    pattern = network_input[start]
    prediction_output = []

    # generate 500 notes
    # 500 개의 노트를 만들어줍니다.
    for note_index in range(500):
        prediction_input = numpy.reshape(pattern, (1, len(pattern), 1))
        prediction_input = prediction_input / float(n_vocab)
# 생성하려는 각 노트에 대해 네트워크에 시퀀스를 제출해야 합니다. 
# 우리가 입력한 첫 번째 시퀀스는 시작 인덱스에 있는 노트 입니다. 
# 우리는 다음 출력 시퀀스를 얻기 위해서, 출력된 시퀀스에서 입력으로 사용된 부분을 제거하고 그것을 입력 시퀀스로 씁니다.


       # 네트워크 출력에서 가장 가능성이 높은 예측값을 결정하기 위해 가장 높은 확률의 인덱스를 추출합니다.    
    # 입력 값에 대해 다음 노트를 예측합니다.
        prediction = model.predict(prediction_input, verbose=0)
        index = numpy.argmax(prediction)
        
        
        
    # 결과값은 숫자가 아닌 노트여야 하므로, 미리 만들어놓은 사전에 숫자를 넣어서 맵핑시킵니다.
        result = int_to_note[index]
        prediction_output.append(result)    #  리스트에 모든 출력을 수집합니다.
        pattern.append(index)
        pattern = pattern[1:len(pattern)]

    return prediction_output

# 이제 리스트에 인코딩된 모든 노트와 코드(chord) 표시가 있으므로 노트의 디코딩을 시작하고 노트와 코드(chord) 개체의 리스트 만들 수 있습니다.

In [ ]:


def create_midi(prediction_output):
    """ convert the output from the prediction to notes and create a midi file
        from the notes """
    offset = 0
    output_notes = []

   
 #먼저 디코딩 중인 출력이 노트인지 또는 코드(chord)인지 확인
    # create note and chord objects based on the values generated by the model
    # 모델에 의해 예측된 값을 바탕으로 노트와 코드(chord) 객체를 만듭니다.
    for pattern in prediction_output:  
        # pattern is a chord
        if ('.' in pattern) or pattern.isdigit():       # 패턴(출력값)이 코드(chord) 일 때
            notes_in_chord = pattern.split('.')         # 문자열을 여러 노트로 분할해야 합니다.
            notes = []
            for current_note in notes_in_chord:         # 그런 다음 각 노트의 문자열 표현을 반복하여 각 노트에 대한 노트 개체를 만듭니다. 
                new_note = note.Note(int(current_note))
                new_note.storedInstrument = instrument.Piano()
                notes.append(new_note)
            new_chord = chord.Chord(notes)            # 그러면 각 노트를 포함하는 코드(chord) 개체를 만들 수 있습니다.
            new_chord.offset = offset
            output_notes.append(new_chord)
        # pattern is a note
        else:      # 패턴이(출력값이) 노트일 때
            new_note = note.Note(pattern)           # 패턴이 노트 인 경우 패턴에 포함된 계이름을 표현하는 문자열 표현을 사용하여 노트 개체를 만듭니다.
            new_note.offset = offset
            new_note.storedInstrument = instrument.Piano()
            output_notes.append(new_note)

            
        # 각 반복마다 오프셋을 0.5 씩 증가시켜 줍니다.
        # 그렇지 않으면 같은 오프셋에 음이 쌓이게 됩니다.    
        # increase offset each iteration so that notes do not stack
        offset += 0.5
        
        
        
# 이제 네트워크에서 생성된 노트 및 코드(chord) 리스트를 매개 변수로 사용하여 Music21 Stream 객체를 만들 것입니다.
# 진짜 음악을 만드는 부분입니다. 
# 마지막으로 네트워크에서 생성된 음악을 저장할 MIDI 파일을 만들기 위해 Music21 툴킷의 쓰기 기능을 사용하여 스트림을 파일에 씁니다.
    midi_stream = stream.Stream(output_notes)

    midi_stream.write('midi', fp='test_output.mid')



In [ ]:

if __name__ == '__main__':
    generate()